In [ ]:
get_ipython().system('pip install transformers datasets scikit-learn pandas torch')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 113.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding, pipeline


In [ ]:
df = pd.read_csv("AirlineTweets.csv")
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
# df = df[['text', 'airline_sentiment']].dropna()
# df = df[df['airline_sentiment'].isin(['positive', 'neutral', 'negative'])]
# df = df.sample(3000, random_state=42).reset_index(drop=True)

In [ ]:
# df['airline_sentiment'].value_counts()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = (
    df[df['airline_sentiment'].isin(['positive', 'neutral', 'negative'])]
    .groupby('airline_sentiment', group_keys=False)
    .apply(lambda x: x.sample(2000, random_state=42))
    .reset_index(drop=True)
)


In [ ]:
df['airline_sentiment'].value_counts()

,count
airline_sentiment,
negative,2000
neutral,2000
positive,2000


In [ ]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['airline_sentiment'])

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


In [ ]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


In [ ]:
train_dataset = Dataset.from_dict({**train_encodings, 'label': train_labels})
val_dataset = Dataset.from_dict({**val_encodings, 'label': val_labels})


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",              # where to save the model
    eval_strategy="epoch",         # evaluate at the end of each epoch
    save_strategy="epoch",               # save at the end of each epoch
    learning_rate=2e-5,                   # standard for fine-tuning
    per_device_train_batch_size=8,        # adjust based on GPU/CPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=3,                   # you can change this
    weight_decay=0.01,                    # regularization
    logging_dir="./logs",                 # for TensorBoard
    logging_steps=50,
    load_best_model_at_end=True,          # load best model after training
    metric_for_best_model="accuracy",     # depends on compute_metrics
    greater_is_better=True,
    report_to="none"                     # if higher metric is better
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)



In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.586400,0.539074,0.782500,0.781630,0.784518,0.782500
2,0.365100,0.594314,0.801667,0.802099,0.803085,0.801667
3,0.273300,0.675880,0.810000,0.809128,0.809019,0.810000


TrainOutput(global_step=1800, training_loss=0.42971268282996283, metrics={'train_runtime': 239.6926, 'train_samples_per_second': 60.077, 'train_steps_per_second': 7.51, 'total_flos': 249622705929600.0, 'train_loss': 0.42971268282996283, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()

# Show evaluation metrics
print("📊 Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")


📊 Evaluation Results:
eval_loss: 0.6759
eval_accuracy: 0.8100
eval_f1: 0.8091
eval_precision: 0.8090
eval_recall: 0.8100
eval_runtime: 2.2376
eval_samples_per_second: 536.2900
eval_steps_per_second: 67.0360
epoch: 3.0000


In [ ]:
model.save_pretrained("fine-tuned-airline-model")
tokenizer.save_pretrained("fine-tuned-airline-model")


('fine-tuned-airline-model/tokenizer_config.json',
 'fine-tuned-airline-model/special_tokens_map.json',
 'fine-tuned-airline-model/vocab.txt',
 'fine-tuned-airline-model/added_tokens.json',
 'fine-tuned-airline-model/tokenizer.json')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tfidf = TfidfVectorizer(max_features=3000)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.74      0.77      0.75       437
           1       0.67      0.67      0.67       380
           2       0.80      0.76      0.78       383

    accuracy                           0.73      1200
   macro avg       0.74      0.73      0.73      1200
weighted avg       0.74      0.73      0.74      1200



In [ ]:
 from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load model and tokenizer manually
model_path = "fine-tuned-airline-model"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# ✅ Fix label mapping (change these according to your training labels)
model.config.id2label = {0: "negative", 1: "neutral", 2: "positive"}
model.config.label2id = {"negative": 0, "neutral": 1, "positive": 2}

# Create pipeline with fixed label mapping
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Predict function
def predict_sentiment(text):
    result = classifier(text, truncation=True)[0]  # truncation to avoid length errors
    label = result['label']
    score = result['score']
    print(f"✈ Sentiment: {label} (Confidence: {score:.2f})")

# Test
test_tweet = input("Enter a tweet to analyze sentiment: ")
predict_sentiment(test_tweet)


Device set to use cuda:0


Enter a tweet to analyze sentiment: My flight departs at 5 PM from Gate 22
✈ Sentiment: neutral (Confidence: 0.98)


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load a high-quality pretrained sentiment model
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create pipeline
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Predict sentiment
def predict_sentiment(text):
    result = classifier(text, truncation=True)[0]
    label = result['label']
    score = result['score']
    print(f"✈ Sentiment: {label} (Confidence: {score:.2f})")

# Test
test_tweet = input("Enter a tweet to analyze sentiment: ")
predict_sentiment(test_tweet)


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


Enter a tweet to analyze sentiment: My flight departs at 5 PM from Gate 22


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✈ Sentiment: neutral (Confidence: 0.93)


In [ ]:
from google.colab import files
!zip -r fine-tuned-airline-model.zip fine-tuned-airline-model
files.download("fine-tuned-airline-model.zip")


  adding: fine-tuned-airline-model/ (stored 0%)
  adding: fine-tuned-airline-model/config.json (deflated 48%)
  adding: fine-tuned-airline-model/tokenizer.json (deflated 71%)
  adding: fine-tuned-airline-model/model.safetensors (deflated 8%)
  adding: fine-tuned-airline-model/vocab.txt (deflated 53%)
  adding: fine-tuned-airline-model/tokenizer_config.json (deflated 75%)
  adding: fine-tuned-airline-model/special_tokens_map.json (deflated 42%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>